<a href="https://colab.research.google.com/github/R12942159/DeepLearning/blob/main/DLCV_hw1_pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch


# Get cuda from GPU device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using: {device}")

In [ ]:
# search file paths
from glob import glob


img_paths_train = sorted(glob('/content/drive/MyDrive/NTU_DLCV/p1_data/train_50/*.png')) # *: all
img_paths_val = sorted(glob('/content/drive/MyDrive/NTU_DLCV/p1_data/val_50/*.png'))

In [ ]:
# number of images
len(img_paths_train), len(img_paths_val)

In [ ]:
import torch
import cv2


class MyDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, img_size):
        self.img_paths = img_paths
        self.img_size = img_size

    def __len__(self):
        """number of samples"""
        return len(self.img_paths)

    def __getitem__(self, idx):
        """read 1 sample"""
        # Read img
        path = self.img_paths[idx] # get img path
        img = cv2.imread(path) # read img
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert to RGB
        img = cv2.resize(img, (self.img_size, self.img_size)) # Resize iamge
        img = img / 255.0 # 0~255 -> 0.~1.

        # Read class index
        cls_idx = int((path.split('/')[-1]).split('_')[0])

        # transform img to tensor
        img = torch.tensor(img, dtype=torch.float)
        img = img.permute(2, 0, 1) # (H, W, C) -> (C, H, W)
        cls_idx = torch.tensor(cls_idx, dtype=torch.long)

        return img, cls_idx

In [ ]:
# Build dataset
IMG_SIZE = 32
train_ds = MyDataset(img_paths_train, IMG_SIZE)
val_ds = MyDataset(img_paths_val, IMG_SIZE)

In [ ]:
# Build DataLoaders
BATCH_SIZE = 64
train_loader = torch.utils.data.DataLoader(train_ds, BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_ds, BATCH_SIZE)